# 데이터 수집
## 웹스크랩핑
    경북대 사회과학기초자료연구소 연구방법논총 논문 다운로드
    대구광역시의회 5분 자유발언
 ## API
     네이버 API, 트위터 API

## 경북대 사회과학기초자료연구소 연구방법논총에서 논문 다운로드
    사이트: http://www.basicdata.kr/journals/ 
    크롬 검사 기능 활용: 크롬→마우스→오른쪽 클릭→검사 

In [ ]:
import wget
url = 'http://www.basicdata.kr/uploads/2018/06/05/04_Lee_2018.pdf'
wget.download(url, '1.pdf')

## c.f. 현재 위치

In [ ]:
import os
os.getcwd()

In [ ]:
url = 'http://www.basicdata.kr/uploads/2018/06/05/03_오수경.신창환_2018.pdf'
wget.download(url, '2.pdf')
url = 'http://www.basicdata.kr/uploads/2018/06/05/02_김진원.정한범_2018.pdf'
wget.download(url, '3.pdf')

## 대구광역시의회 5분자유발언 수집
    사이트: https://council.daegu.go.kr/kr/activity/free
    수집항목: 대수, 회의명, 의원명, 제목, 질문일 
    출처: 경북대학교 김창현  

In [ ]:
from bs4 import BeautifulSoup
import requests

i=1
num=range(i,10)

for i in num :
    url = 'https://council.daegu.go.kr/kr/activity/free?reform=list&page={}&flag=&keyword=&search_code=&th_sch=all&th_sch=all&mem_sch=&sess_sch='.format(i)
    req=requests.get(url, verify=False)
    html = req.content
    soup = BeautifulSoup(html, 'html.parser')
    for link in soup.find_all('td'):
        data=link.get_text('td')
        print(data)
    i +=1

In [ ]:
from bs4 import BeautifulSoup
import requests

i=1
num=range(i, 2)
with open('daegu.txt', 'w', encoding='UTF8') as f:
    for i in num:
        url = 'https://council.daegu.go.kr/kr/activity/free?reform=list&page={}&flag=&keyword=&search_code=&th_sch=all&th_sch=all&mem_sch=&sess_sch='.format(i)
        req=requests.get(url, verify=False)
        html = req.content
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('td'):
            data=link.get_text('td')
            print(data)
            f.write(data)
        i +=1


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

i=1
num=range(i, 2)
with open('daegu.csv', 'w', encoding='UTF8') as f:
    csv_writer=csv.writer(f)
    for i in num:
        url = 'https://council.daegu.go.kr/kr/activity/free?reform=list&page={}&flag=&keyword=&search_code=&th_sch=all&th_sch=all&mem_sch=&sess_sch='.format(i)
        req=requests.get(url, verify=False)
        html = req.content
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('td'):
            data=link.get_text('td')
            csv_writer.writerow([data]) 
        i +=1

## 네이버 API를 통한 신문기사 데이터 수집
    네이버 애플리케이션 등록: https://developers.naver.com/apps/#/register
    신문기사 검색: 사용 API → 검색

In [ ]:
import requests
import json

with open('naver_config.json') as f:
    tokens = json.loads(f.read())

Client_ID = tokens["Client_ID"]
Client_Secret = tokens["Client_Secret"]

query = '경북대학교'
url = 'https://openapi.naver.com/v1/search/news.json?query={}&display=1'.format(query)
headers = {
    'X-Naver-Client-Id': Client_ID,
    'X-Naver-Client-Secret': Client_Secret,
           }
req = requests.get(url, headers=headers)
# print(req.text)
json_data = json.loads(req.text)
# print(json_data)

for data in json_data['items']:
    print("기사제목: {0}, 기사링크 : {1}, 기사내용: {2}, 기사날짜: {3}".format(data['title'],
                                                                                data['originallink'],
                                                                                data['description'],
                                                                                data['pubDate']))

## 트위터 데이터 수집
### 트위터 수집 방법

    Real time tweets → Streaming API: tweepy
    Historical tweets →  Search API (1 week): twittersearch
    User networks → Followers API
    Geolocation → Geo API
https://dev.twitter.com/rest/reference/get/geo/search

#### c.f. 쥬피터 내부에서 패키지 인스톨

In [ ]:
!pip3 install TwitterSearch 
import TwitterSearch

In [ ]:
from TwitterSearch import *
import time
import datetime
import json
import csv

In [ ]:
######### Twitter auth 호출 #################
with open('twitter_config.json') as f:
    tokens = json.loads(f.read())
#######################################

In [ ]:
consumer_key = tokens["consumer_key"]
consumer_secret = tokens["consumer_secret"] # DO NOT SHARE WITH ANYONE!
access_token = tokens["access_token"]
access_token_secret = tokens["access_token_secret"]

In [ ]:
file_id = 'korea'
now_date = str(datetime.date.today())

try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['한국', '대한민국', '남한'], or_operator=True) # let's define all words we would like to have a look for
    tso.set_language('ko') # we want to see Korean tweets only
    tso.set_include_entities(False) # and don't give us all those entity information
    # tso.set_until(datetime.date(2017, 4, 1))
    # tso.set_since_id(25007592717275955) # id after first collection

    # it's about time to create a TwitterSearch object with our secret tokens
    ts = TwitterSearch(
        consumer_key = consumer_key,
        consumer_secret = consumer_secret,
        access_token = access_token,
        access_token_secret = access_token_secret
    )

     # this is where the fun actually starts :)
    with open('{0}_TwitterSearch_{1}.csv'.format(file_id, now_date), 'w', encoding='utf-8') as saveFile:
        csv_writer = csv.writer(saveFile, delimiter='\t')
        csv_writer.writerow(['tweet_time', 'twitter_id', 'tweet_text'])
        for tweet in ts.search_tweets_iterable(tso):
            #print( '%s tweeted: %s on %s' % ( tweet['user']['id_str'], tweet['text'], tweet['created_at'] ) )
            tweet_time = time.strftime('%Y-%m-%d', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
            print(tweet_time)
            twitter_id = tweet['user']['id_str']
            print(twitter_id)
            tweet_text = tweet['text'].replace("\n", "")
            print(tweet_text)
            csv_writer.writerow([tweet_time, twitter_id, tweet_text])
       
except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

2018-08-17
950313850196209665
RT @C_F_diablesse: 모든 걸 &lt;입법&gt;의 문제로만 돌리면 한국에 “사법부”가 왜 필요하며, 법관의 “재량”을 존중해야 할 까닭은 무엇이겠습니까? 이번 “안희정 성폭력 사건”은 현행법의 테두리 안에서도 유죄의 인정이 가능했던 사건이었음에도…
2018-08-17
565416806
RT @space_sauna: 시원한 바람에 놀란 한국인들 한 2개월 뒤엔 한파에 놀랄것이다
2018-08-17
141585942
RT @Linco_a: 단백질 히잡이라며 긴머리 잘라내서 탈코하는 것도 결국 남성시선에서 여성성을 부여하는 20-30대 젋은 여성에게나 탈코인거지..(물론 나잇대로 딱 분류할 수 있는 문제는 아님) 예전에도 얘기했는데 한국의 중년 이후 여성의 디폴트…
2018-08-17
1016064858645618688
RT @JIN_Ji92: 후... 지민 허리춤에 저거.. 인제 앤써 음원은 캐신나고 행복할거라 이말입니다,,, 한국..컴백..끝내버리겠다는 정국의 말,, 뼈있는 말이다 이겁니다ㅠㅠㅠㅠㅠㅠㅠㅠㅠ♡#FFGroupBTS #MMVAs #iHeartR…
2018-08-17
1007679628817850369
RT @kogasa_parang: 이제 외국인들에게 한국은 매일 밤늦게까지 공부를 하고 야식으로 익스트림 스파이시 누들을 한 그릇씩 해치우는 미친 나라로 각인된 듯 (놀랍게도 사실이다) https://t.co/myhTBYipno
2018-08-17
977900687907479552
RT @re_d27: 애인님 : 한국 올때 "누가 이런걸 사?" 싶은 괴식 있으면 좀 사와봐.나 : 알았어. https://t.co/o8HOf3zZhH
2018-08-17
3988597403
RT @beluga2046: 지금 BBC 라디오에서 '세계 1위의 저출산률 국가인 한국'에 대한 방송이 나오고 있다. 원인은 1) 임신하고 아이를 가진 여사원에 대한 회사의 퇴사 압력 2) 아이 돌보는 사회적 인프라 부족 

2018-08-17
367138934
RT @rassak: 왜 한국 매체들은 죄다 남성 범죄자의 변명을 헤드라인으로 실어주죠. 홍대 불법촬영 기사에선 "성기를 드러내고 휴게실을 혼자 점령한 남성에게 욱하는 마음에... 반성문도 7차례나 보냈으나..." 같은 변명 안 실어줬을 거 아니에요…
2018-08-17
43074550
RT @lady_sullem: 에휴 어디서 모자란 한국 남자가 와서 여성 혐오 욕하는 건 하루 이틀 아니어서 웃긴데페미니즘한다는 한국 여자들한테 욕먹는 건 너무 힘들다. 커뮤 끊을때가 된듯.비혼 비출산 응원하지만, 기혼했다고, 육아한다고 한남 품…
2018-08-17
869051234350542850
RT @HjulesY: 한국 여자로 살면서 나만 미친년 될 것 같아서 가만히 있어본 경험 없는 사람이 누가 있을까. 그런데 누가 전국구 미친년으로 남을 걸 각오하고 목소리를 냈다. 지금까지의 자기 삶에는 영원히 작별을 고해야 할지도 모르는데. 어떻게…
2018-08-17
873909653184237568
RT @netsukei: 한국인이 필리핀에서 3년사이 31명 피살당함 -&gt; 이유 : 안에서 새던 바가지 밖에서도 새서야 너희들 거 한국여자들이 총 안드는 거 고맙게 여기라우 https://t.co/tNAaVIdqUl
2018-08-17
1020994881500545025
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
4786255771
RT @KDH_official: 도쿄-나고야-오사카를 거쳐 다시 도쿄로, 일본에서 진행된 첫 쇼케이스&amp;팬미팅의 긴 여정에 함께해주신 팬분들께 감사드립니다. 함께 했던 시간이 많이 행복해 오랫동안 기억에 남을 것 같아요. 한국 팬미팅에서 만나요!…
2018-08-17
553287530
RT @nadal_imsi: '김성태 나와라!' 국회 찾아간 상인들 / MediaVOP민생 외면하는 자유한국당과 김성태에 뿔난 중소상인들"국회 계류 

2018-08-17
553287530
RT @bleuchatte: 같은 매크로 여론조작 의혹인데, 언론이 외면하는 사건 https://t.co/1s9y2nI8pl김경수-드루킹 보도에는 매달리면서 한나라당 매크로는 무관심한 언론자유한국당이라는 정당이 조직적으로 여론조작한 건 개무시…
2018-08-17
235048064
RT @rtk_news: [알티케이뉴스] 한국, 말레이시아전에서 16강 조기 진출 노려.. 손홍민 출장 예상 (출처 : RTK 뉴.. | https://t.co/U2xpUfy2li 블로그) https://t.co/bPObTzJkee
2018-08-17
3273110899
메렌 성대가 하필 제 한국 차애 성우님이라서요...
2018-08-17
426996386
RT @lawsooni: 한국은 대학이라는 시스템을 수입했을 뿐이지 그에 준하는 콘텐츠를 만들어내지 못했기 때문에 권위주의만 있고 실제 권위가 없어요. 교수 수준이 사회의 평균적 교양에도 못 미치는 경우도 꽤 되는데 이 교수가 그렇죠. https:/…
2018-08-17
2706136824
RT @beluga2046: 지금 BBC 라디오에서 '세계 1위의 저출산률 국가인 한국'에 대한 방송이 나오고 있다. 원인은 1) 임신하고 아이를 가진 여사원에 대한 회사의 퇴사 압력 2) 아이 돌보는 사회적 인프라 부족 3) 아이 한명 키우는데 시…
2018-08-17
706639358393061376
RT @hejsommer: 덴마크인 친구가 이 링크 보내주면서 기자가 극단적으로 묘사했다길래 어디가 극단적인지 한국여자로서 지극히 논리적이라 생각한다고 함 BBC News - 'Why I never want babies' https://t.co/y…
2018-08-17
921143556101758976
RT @akckd0207: [카드뉴스] 대한민국 정부는 왜 강제개종에 침묵하나 https://t.co/cgyAa39i15 …#강제개종  #한기총 #구지인법 #CBS폐쇄 #구지인추모 https://t.co/g

2018-08-17
2942790984
RT @DanYeol_cos: 한국날씨 현재 상황 https://t.co/6X3JTdHK9s
2018-08-17
4801965374
RT @kogasa_parang: 이제 외국인들에게 한국은 매일 밤늦게까지 공부를 하고 야식으로 익스트림 스파이시 누들을 한 그릇씩 해치우는 미친 나라로 각인된 듯 (놀랍게도 사실이다) https://t.co/myhTBYipno
2018-08-17
2412444637
RT @beluga2046: 지금 BBC 라디오에서 '세계 1위의 저출산률 국가인 한국'에 대한 방송이 나오고 있다. 원인은 1) 임신하고 아이를 가진 여사원에 대한 회사의 퇴사 압력 2) 아이 돌보는 사회적 인프라 부족 3) 아이 한명 키우는데 시…
2018-08-17
1004654077207179264
RT @_2mong: 한국남자는 가만히 앉아있으면 여자들이 9첩 밥상을 차려오고, 집가서 엄마 밥줘 한다는 걸 정확히 이해하다니 여윽시 한국 항공사 진에어 ~ https://t.co/aeoUw57d6m
2018-08-17
320715866
RT @space_sauna: 시원한 바람에 놀란 한국인들 한 2개월 뒤엔 한파에 놀랄것이다
2018-08-17
882516032119635968
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
858653556399788032
RT @DanYeol_cos: 한국날씨 현재 상황 https://t.co/6X3JTdHK9s
2018-08-17
897469124074328065
RT @geometryofsleep: 미국에 &lt;To catch a predator&gt; 이라는 방송이 있었다. 이게 뭐냐면 “온라인으로 미성년자에게 접근해 섹스하는 성범죄자들”을 잡는 프로. 탐라에 한국남자가 검거되었을 때 영상이 올라와서 하는 말인데…
2018-08-17
1614573798
RT @syk99144: @oxlxlo "아따 여가 그 

2018-08-17
813520794923868160
RT @silviasun1122: 한국이 얼마나 싫었으면 ㅅㅂ직각으로꺾어버리네;;안기준보다운전잘함 https://t.co/4mGwyyY1A4
2018-08-17
3476687184
RT @hejsommer: 덴마크인 친구가 이 링크 보내주면서 기자가 극단적으로 묘사했다길래 어디가 극단적인지 한국여자로서 지극히 논리적이라 생각한다고 함 BBC News - 'Why I never want babies' https://t.co/y…
2018-08-17
831475676750479360
RT @beluga2046: 지금 BBC 라디오에서 '세계 1위의 저출산률 국가인 한국'에 대한 방송이 나오고 있다. 원인은 1) 임신하고 아이를 가진 여사원에 대한 회사의 퇴사 압력 2) 아이 돌보는 사회적 인프라 부족 3) 아이 한명 키우는데 시…
2018-08-17
715890796352778240
RT @Di_an27: 한국에 있을 땐 진짜 이상한 말 많이 들었는데 https://t.co/wHfB893EQp
2018-08-17
4668762673
RT @kogasa_parang: 이제 외국인들에게 한국은 매일 밤늦게까지 공부를 하고 야식으로 익스트림 스파이시 누들을 한 그릇씩 해치우는 미친 나라로 각인된 듯 (놀랍게도 사실이다) https://t.co/myhTBYipno
2018-08-17
973117524895154177
RT @fuckin_moon: Rt)저 탈한국이 너무 간절해서 독학으로 유학 준비를 하고 있어요. 사실 곧 시험인데 준비한게 하나도 없어서 마지막 기회라 열심히 해야되요.ㅜㅜ 만약에 다음달에 보는 시험 및 면접에 붙게 된다면 아래 것들 추첨해서 한…
2018-08-17
1019264593124683776
RT @yoonRest_js: 8월13일(월), 지성님이 한국으로 귀국할 때 착용했던 비코뱃지를 기억하시나요?9월10일 워너원 데뷔400일 기념으로 비코뱃지를 공동구매합니다. 입금기한:~9월9일23시59초

...

2018-08-17
3224444286
RT @Sielrin_CM: 2018.08.17 업데이트 안내《가을 패치》- 가을이 시작됩니다.- 업데이트는 지역마다 차이가 있으므로 유의해주시기 바랍니다.- 올 해는 대한민국 서버가 개설된 지 4351 년입니다.- 패치 전과 패치 후…
2018-08-17
984391782191558656
RT @heezBK: 고3 한국고딩 쿠고 https://t.co/rW3GDQrnCI
2018-08-17
837969483637657600
RT @youthdonation: 박수영 너무 웃기다 진짜 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 한국에 있다가 슬로베니아가서 감탄 중 ㅋㅋㅋㅋㅋㅋㅋㅋ https://t.co/aU2Fc2cEFm
2018-08-17
409625273
RT @space_sauna: 시원한 바람에 놀란 한국인들 한 2개월 뒤엔 한파에 놀랄것이다
2018-08-17
755405602197610497
RT @TheNextpeople: 2004년 오늘(8월17일) 노무현 대통령, 한국기자협회 창립 40주년 기념식 참석"진실은 국익의 앞에 섭니다. 그러나 같은 뜻으로 진실은 기자의 감정이나 이해관계에도 우선해야 하고 언론사와 그 시기에 특별히 힘을…
2018-08-17
1025748278694961152
RT @fuckin_moon: Rt)저 탈한국이 너무 간절해서 독학으로 유학 준비를 하고 있어요. 사실 곧 시험인데 준비한게 하나도 없어서 마지막 기회라 열심히 해야되요.ㅜㅜ 만약에 다음달에 보는 시험 및 면접에 붙게 된다면 아래 것들 추첨해서 한…
2018-08-17
817558911972540417
RT @yeon_pa_rang: 부장님 친구분이 굉장한 엘리트라 2년은 한국,3년은 미국에서 돌아가며 근무한다며 만나볼라냐고 자꾸 물으신다.자산 규모가 300억 정도 된다고.그런 분이 굳이 저를 만나시겠냐고 하니 "미국에서 밥해줄 사람도 없이 혼자…
2018-08-17
2776264184
RT @DanYeol_cos: 한국날씨 현재 상황 https:/

2018-08-17
1647711950
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
1490531311
RT @worldwright: RT) 이게 의도적인지 아닌지 모르겠는데, 한국에서는 새로운 용어를 기득권이 전복적으로 사용하는 경우가 많다. 묘하게 교활하다. “강간 사건”이 “미투 사건”이 되더니, “성적 자기결정권”이 보호될 권리가 아니라 피해자…
2018-08-17
872366144186970113
RT @busanHE: 오늘 한국의 말종들 태극기집회를 보고 올린 댓글중 쵝오의 명언(?)"아빠는 태극기 엄마는 엄마부대 아들은 일베 딸은 워마드 즐거운 우리집 ㅋㅋㅋ"
2018-08-17
983316636609347584
RT @kogasa_parang: 이제 외국인들에게 한국은 매일 밤늦게까지 공부를 하고 야식으로 익스트림 스파이시 누들을 한 그릇씩 해치우는 미친 나라로 각인된 듯 (놀랍게도 사실이다) https://t.co/myhTBYipno
2018-08-17
397724020
RT @rassak: 왜 한국 매체들은 죄다 남성 범죄자의 변명을 헤드라인으로 실어주죠. 홍대 불법촬영 기사에선 "성기를 드러내고 휴게실을 혼자 점령한 남성에게 욱하는 마음에... 반성문도 7차례나 보냈으나..." 같은 변명 안 실어줬을 거 아니에요…
2018-08-17
957755548161617920
RT @dripping_bik: 어쩐지 한국에서는 집에 걸어두면 돈 들어온다는 그림이 되버린 구스타브 클림트의 작품들,,, #rt로_몇명이_이작품을_아는지_알아보자 https://t.co/gFP74Y0bB5
2018-08-17
139768099
홍경민횽 이번주도 연속 등판 !!!역시 한국의 리키마틴~~|
2018-08-17
1010826227417075713
RT @cryptologic888: 대한민국 NO.1 놀이터 회원모집합니다.10년 이상 운영되고 있는 신규 우리계열 사이트입니다.먹튀 걱정 NO~~블

2018-08-17
729592135511498753
RT @Di_an27: 한국에 있을 땐 진짜 이상한 말 많이 들었는데 https://t.co/wHfB893EQp
2018-08-17
790841145274142721
RT @NextSprings: 여자가 한국에서 숏컷이면 남자로 생각되고 미국에서 숏컷이면 레즈비언이라 생각하는 거 블랙 코미디같다
2018-08-17
431275886
RT @ldccd: @newstapa 대한민국은 넘팽들에 천국입니다
2018-08-17
805292322820763648
RT @scjbong144: [오해와진실] #국민들께 부패한 한국기독교총연합회(한기총)을 고발합니다! https://t.co/RZCe42K5PE https://t.co/EejmfPBnMS
2018-08-17
761108846907564032
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
474473212
RT @RealVIXX: 나는 로빅이다. #레오 요원이 출연하는 #심플리케이팝 이 잠시 후 오후 1시 해외 아리랑 월드 채널에서 방송된다. 한국 아리랑 채널에서는 토요일 밤 11시에 만나볼 수 있다! [이상, RT작전 실행하라!]#빅스 #VIXX…
2018-08-17
218690884
RT @baryu43: 3. 정부,금융기관에서 설치강요한 키보드보안, 액티브엑스 제거한국이 더러운 이유중에 하나입니다. 외국사이트에선 1도 요구 안하는 구라들을 자꾸 설치하라해요 ㅡㅡ 그럴때 이걸 쓰십시오https://t.co/H3ThMT…
2018-08-17
3163366470
RT @Sielrin_CM: 2018.08.17 업데이트 안내《가을 패치》- 가을이 시작됩니다.- 업데이트는 지역마다 차이가 있으므로 유의해주시기 바랍니다.- 올 해는 대한민국 서버가 개설된 지 4351 년입니다.- 패치 전과 패치 후…
2018-08-17
983192295846567936
RT @sangchoo9319: https:

2018-08-17
998746890828038145
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
2452682215
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
875369871638372352
RT @beluga2046: 지금 BBC 라디오에서 '세계 1위의 저출산률 국가인 한국'에 대한 방송이 나오고 있다. 원인은 1) 임신하고 아이를 가진 여사원에 대한 회사의 퇴사 압력 2) 아이 돌보는 사회적 인프라 부족 3) 아이 한명 키우는데 시…
2018-08-17
701794762454540292
RT @DanYeol_cos: 한국날씨 현재 상황 https://t.co/6X3JTdHK9s
2018-08-17
1457589259
RT @girin25: 백합러중에 젤 무서운 새끼들...그건 한국 드라마로 백합 착즙하는 새끼들이다...이들은 스스로가 착즙하는 줄도 잘 모르고 한국식뼈테로엔딩나도 계속 백합을 착즙함...모래에서 기름을 짜고 슈루탄으로 남주를 터트릴 놈들...
2018-08-17
857927586965475329
RT @jongwon1917: 남한을 찾는 러시아 관광객 증가와 함께 러시아 여성 커뮤니티에선 "молька" [몰카] 주의를 당부하는 자료들도. https://t.co/eCeX8az28u
2018-08-17
960038963418251264
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
955357285454249984
RT @silviasun1122: 한국이 얼마나 싫었으면 ㅅㅂ직각으로꺾어버리네;;안기준보다운전잘함 https://t.co/4mGwyyY1A4
2018-08-17
858000016660586497
RT @DanYeol_cos: 한국날씨 현재 상황 https://t.co/6X3JTdHK9s
2018-08-17
729

2018-08-17
788686356285272064
RT @un_0513: 다시ㅣ 보는 답 없는 한국... https://t.co/K3FYuPNCKy
2018-08-17
941980030975881216
RT @youthdonation: 박수영 너무 웃기다 진짜 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 한국에 있다가 슬로베니아가서 감탄 중 ㅋㅋㅋㅋㅋㅋㅋㅋ https://t.co/aU2Fc2cEFm
2018-08-17
1000217503514021894
RT @BVIP_1116: 승리야...네 생일만큼은 한국에서 보내는게 어때..?작년에 영배 지용 대성이 다 일본에서 축하했잖아...이제는 좀 한국VIP가 축하해주고싶어... https://t.co/IIDujjssCB
2018-08-17
981578586619432960
RT @beluga2046: 지금 BBC 라디오에서 '세계 1위의 저출산률 국가인 한국'에 대한 방송이 나오고 있다. 원인은 1) 임신하고 아이를 가진 여사원에 대한 회사의 퇴사 압력 2) 아이 돌보는 사회적 인프라 부족 3) 아이 한명 키우는데 시…
2018-08-17
987647773158723585
RT @forn_official: [RT이벤트✨]홍콩 SASA 입점해서 기부니가 좋으니까!!RT 20명 👉 지우개패드or겨드랑이패드(랜덤증정)를 드립니다💕홍콩 진출한 지패,겨패 한국에서 써보실 분~?😆당첨자 발표는 8월 22일입니다🍀 ht…
2018-08-17
746616315352252420
RT @Sielrin_CM: 2018.08.17 업데이트 안내《가을 패치》- 가을이 시작됩니다.- 업데이트는 지역마다 차이가 있으므로 유의해주시기 바랍니다.- 올 해는 대한민국 서버가 개설된 지 4351 년입니다.- 패치 전과 패치 후…
2018-08-17
730069375785426944
RT @yes02170: 한국기독교총연합회는 진정 하나님을 믿고 따르는 곳인가???예수님과 칼빈, 신천지 교주vs한기총 교주 고발 진실공방https://t.co/ScSiq5kciX #한기총_고

#### 출처 : https://github.com/ckoepp/TwitterSearch, 방문일 2017년 3월 13일